In [12]:
import cv2
import os
from time import time
import glob
import h5py
import pandas as pd

In [43]:
os.chdir("/home/donghan/DeepLabCut/data/")
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[260:] 

In [65]:
def videoWriter(task, filename, outputName, corrX, corrY, distX, distY, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (distY, distX)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out
    
def videoEditer(filename, outputName, corrX, corrY, distX, distY, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + distX
    corrY1 = corrY + distY

    video = videoWriter("edited", filename, outputName, corrX, corrY, distX, distY, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

    
def getCoord(data):
    '''
    Task: Obtaining the coordinates of cropped points
    '''
    xMax = max(round(data['x']).astype(int))
    xMin = min(round(data['x']).astype(int))
    yMax = max(round(data['y']).astype(int))
    yMin = min(round(data['y']).astype(int))
    
    lengthX = xMax - xMin
    lengthY = yMax - yMin
    
#     if yMax - yMin < xMax - xMin:
#         length = yMax - yMin
#     else:
#         length = xMax - xMin
    
    
    return xMin, yMin, lengthX, lengthY



In [48]:
videoEditer('1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest.mp4", 10,10,300)

1034 SI_B, Aug 15, 13 7 49.mp4 successfully  edited!


In [63]:
x, y, lengthX, lengthY = getCoord(move_data)

In [66]:
videoEditer('1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest.mp4", y, x, lengthX, lengthY)

1034 SI_B, Aug 15, 13 7 49.mp4 successfully  edited!


In [21]:
head = list(zip(round(move_data['x']).astype(int), round(move_data['y']).astype(int)))
tail = list(zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int)))

In [23]:

sorted([i[0] for i in head], reverse = True)

[435,
 424,
 390,
 389,
 389,
 389,
 388,
 387,
 387,
 387,
 387,
 386,
 386,
 386,
 385,
 385,
 385,
 385,
 384,
 384,
 384,
 382,
 382,
 382,
 382,
 381,
 380,
 380,
 379,
 379,
 379,
 379,
 378,
 378,
 378,
 378,
 378,
 377,
 377,
 377,
 377,
 376,
 376,
 376,
 376,
 376,
 375,
 375,
 375,
 375,
 375,
 375,
 375,
 375,
 374,
 374,
 374,
 374,
 374,
 374,
 374,
 374,
 374,
 373,
 373,
 373,
 373,
 373,
 373,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 371,
 371,
 371,
 371,
 371,
 371,
 371,
 371,
 371,
 371,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 369,
 369,
 369,
 369,
 369,
 369,
 369,
 369,
 369,
 369,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 367,
 367,
 367,
 367,
 367,
 367,
 367,
 367,
 367,
 367,
 367,
 366,
 366,
 366,
 366,
 366,
 366,
 366,
 366,
 366,
 365,
 365,
 365,
 365,
 365,
 364,
 364,
 364,
 364,
 364,
 364

In [15]:
max(head)

(448, 298)

In [5]:
video = videoWriter("crop", '1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest.mp4")
cap = video[0]
out = video[1]


Successfully created the directory ./crop 
